In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet.gluon import nn

import h5py
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

ctx = mx.gpu()

In [2]:
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter_val)
    for data, label in data:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [3]:
with h5py.File('features.h5', 'r') as f:
    features_vgg = np.array(f['vgg'])
    features_resnet = np.array(f['resnet'])
    features_densenet = np.array(f['densenet'])
    features_inception = np.array(f['inception'])
    labels = np.array(f['labels'])

In [4]:
features_resnet = features_resnet.reshape(features_resnet.shape[:2])
features_inception = features_inception.reshape(features_inception.shape[:2])

features = np.concatenate([features_resnet, features_densenet, features_inception], axis=-1)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(features, labels)

dataset_train = gluon.data.ArrayDataset(nd.array(X_train), nd.array(y_train))
dataset_val = gluon.data.ArrayDataset(nd.array(X_val), nd.array(y_val))

batch_size = 128
data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)

In [6]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dropout(0.5))
    net.add(nn.Dense(120))

net.initialize(ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 1e-4, 'wd': 1e-5})
net

Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dropout(p = 0.5)
  (2): Dense(None -> 120, linear)
)

In [7]:
for epoch in range(30):
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    val_loss, val_acc = evaluate(net, data_iter_val)
    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100))

Epoch 1. loss: 4.6348, acc: 4.92%, val_loss 3.9883, val_acc 23.46%
Epoch 2. loss: 3.5632, acc: 23.09%, val_loss 2.6124, val_acc 61.97%
Epoch 3. loss: 2.4211, acc: 46.51%, val_loss 1.5162, val_acc 78.45%
Epoch 4. loss: 1.6728, acc: 60.84%, val_loss 1.0034, val_acc 83.65%
Epoch 5. loss: 1.2647, acc: 69.35%, val_loss 0.7494, val_acc 85.72%
Epoch 6. loss: 1.0422, acc: 73.79%, val_loss 0.6181, val_acc 87.05%
Epoch 7. loss: 0.8917, acc: 77.17%, val_loss 0.5362, val_acc 88.38%
Epoch 8. loss: 0.7709, acc: 80.29%, val_loss 0.4820, val_acc 88.46%
Epoch 9. loss: 0.6899, acc: 81.59%, val_loss 0.4492, val_acc 88.93%
Epoch 10. loss: 0.6352, acc: 82.74%, val_loss 0.4185, val_acc 88.97%
Epoch 11. loss: 0.5687, acc: 84.11%, val_loss 0.4027, val_acc 89.40%
Epoch 12. loss: 0.5438, acc: 84.98%, val_loss 0.3859, val_acc 89.40%
Epoch 13. loss: 0.4938, acc: 86.28%, val_loss 0.3721, val_acc 89.40%
Epoch 14. loss: 0.4673, acc: 86.43%, val_loss 0.3647, val_acc 89.40%
Epoch 15. loss: 0.4432, acc: 87.38%, val_los

In [8]:
with h5py.File('features_test.h5', 'r') as f:
    features_vgg_test = np.array(f['vgg'])
    features_resnet_test = np.array(f['resnet'])
    features_densenet_test = np.array(f['densenet'])
    features_inception_test = np.array(f['inception'])

In [9]:
features_resnet_test = features_resnet_test.reshape(features_resnet_test.shape[:2])
features_inception_test = features_inception_test.reshape(features_inception_test.shape[:2])

features_test = np.concatenate([features_resnet_test, features_densenet_test, features_inception_test], axis=-1)

In [10]:
output = nd.softmax(net(nd.array(features_test).as_in_context(ctx))).asnumpy()

In [11]:
df = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df.columns[1:]):
    df[c] = output[:,i]

df.to_csv('pred.csv', index=None)